In [1]:
import pandas as pd
import numpy as np
import ipywidgets as widgets
from IPython.display import display, clear_output
from sklearn.metrics import (
    mean_absolute_error, mean_squared_error, r2_score,
    precision_score, recall_score, f1_score, roc_auc_score,
    confusion_matrix
)
from openpyxl import Workbook, load_workbook

# Function to calculate regression metrics
def calculate_regression_metrics(y_true, y_pred):
    mae = mean_absolute_error(y_true, y_pred)
    mse = mean_squared_error(y_true, y_pred)
    rmse = np.sqrt(mse)
    r2 = r2_score(y_true, y_pred)
    n = len(y_true)
    adj_r2 = 1 - (1 - r2) * (n - 1) / (n - 1)
    return {
        'Metric': ['MAE', 'MSE', 'RMSE', 'R2', 'Adjusted R2'],
        'Value': [mae, mse, rmse, r2, adj_r2]
    }

# Function to calculate classification metrics
def calculate_classification_metrics(y_true, y_pred):
    precision = precision_score(y_true, y_pred)
    recall = recall_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred)
    auc = roc_auc_score(y_true, y_pred)
    gini = 2 * auc - 1
    confusion = confusion_matrix(y_true, y_pred)

    metrics = {
        'Metric': ['Precision', 'Recall', 'F1-Score','AUC ROC Score', 'GINI'],
        'Value': [precision, recall, f1, auc, gini]
    }

    return metrics, confusion

# Function to calculate business metrics
def calculate_business_metrics(y_true, probabilities, amount, gdr_threshold):
    df = pd.DataFrame({
        'y_true': y_true,
        'probabilities': probabilities,
        'amount': amount
    })
    df = df.sort_values(by='probabilities', ascending=False).reset_index(drop=True)

    df['cum_1s'] = df['y_true'].cumsum()
    df['cum_amount_1s'] = df['amount'] * df['y_true']
    df['cum_0s'] = (~df['y_true'].astype(bool)).cumsum()

    df['TDR'] = df['cum_1s'] / df['y_true'].sum()
    df['VDR'] = df['cum_amount_1s'].cumsum() / (df['amount'] * df['y_true']).sum()
    df['GDR'] = df['cum_0s'] / (~df['y_true'].astype(bool)).sum()

    selected_row = df[df['GDR'] <= gdr_threshold].iloc[-1]

    return {
        'Metric': ['TDR', 'VDR'],
        'Value': [selected_row['TDR'], selected_row['VDR']]
    }

# Function to save regression results to Excel
def save_regression_results_to_excel(results, dataset_type):
    file_name = 'regression_results.xlsx'
    try:
        # Try to load the existing workbook
        workbook = load_workbook(file_name)
    except FileNotFoundError:
        # If not found, create a new workbook and remove the default sheet
        workbook = Workbook()
        if 'Sheet' in workbook.sheetnames:
            del workbook['Sheet']

    # Check if the sheet for the dataset type already exists, and create if not
    if dataset_type not in workbook.sheetnames:
        workbook.create_sheet(title=dataset_type)
    
    # Select the appropriate sheet
    sheet = workbook[dataset_type]

    # Append the header only if it's a new sheet
    if sheet.max_row == 1:
        sheet.append(['Metric', 'Value'])

    # Append the new results
    for metric, value in zip(results['Metric'], results['Value']):
        sheet.append([metric, value])

    # Save the workbook
    workbook.save(file_name)
    print(f'Regression results saved to {file_name}, sheet: {dataset_type}')

# Function to save classification results to Excel
def save_classification_results_to_excel(results, confusion, business_metrics, dataset_type):
    file_name = 'classification_results.xlsx'
    try:
        workbook = load_workbook(file_name)
    except FileNotFoundError:
        workbook = Workbook()
        if 'Sheet' in workbook.sheetnames:
            del workbook['Sheet']

    if dataset_type not in workbook.sheetnames:
        workbook.create_sheet(title=dataset_type)

    sheet = workbook[dataset_type]

    if sheet.max_row == 1:
        sheet.append(['Metric', 'Value'])

    if 'Metric' in results and 'Value' in results:
        for metric, value in zip(results['Metric'], results['Value']):
            sheet.append([metric, value])

    if len(confusion) > 0:
        sheet.append([])
        sheet.append(['Confusion Matrix'])

        confusion_df = pd.DataFrame(confusion, index=['Actual 0', 'Actual 1'], columns=['Predicted 0', 'Predicted 1'])
        header = [''] + confusion_df.columns.tolist()
        sheet.append(header)

        for idx, row in confusion_df.iterrows():
            sheet.append([idx] + row.tolist())

    if 'Metric' in business_metrics and 'Value' in business_metrics:
        sheet.append([])
        sheet.append(['Business Metrics'])

        for metric, value in zip(business_metrics['Metric'], business_metrics['Value']):
            sheet.append([metric, value])

    workbook.save(file_name)
    print(f'Classification results saved to {file_name}, sheet: {dataset_type}')

# Main function for widget-based interface
def create_widgets_for_metrics(dataset):
    global current_dataset

    dataset_type = widgets.Dropdown(
        options=[('Select One', 'select_one'), ('Train', 'train'), ('Test', 'test')],
        description='Dataset Type:',
    )

    def on_dataset_type_change(change):
        global current_dataset
        clear_output(wait=True)
        display(dataset_type)
        if change['new'] == 'train':
            current_dataset = dataset.copy()
            display(train_buttons)
        elif change['new'] == 'test':
            current_dataset = dataset.copy()
            display(test_buttons)

    dataset_type.observe(on_dataset_type_change, names='value')

    regression_button = widgets.Button(description='Regression Metrics', tooltip="Generate metrics for regression tasks.")
    classification_button = widgets.Button(description='Classification Metrics', tooltip="Generate metrics for classification tasks.")

    def on_regression_button_clicked(b):
        regression_button.style.button_color = 'lightblue'  # Highlight the selected button
        classification_button.style.button_color = None  # Reset the other button color
        clear_output(wait=True)
        display(dataset_type)
        display(regression_button, classification_button)  # Re-display the buttons
        display(regression_selection)
    
    def on_classification_button_clicked(b):
        classification_button.style.button_color = 'lightblue'  # Highlight the selected button
        regression_button.style.button_color = None  # Reset the other button color
        clear_output(wait=True)
        display(dataset_type)
        display(regression_button, classification_button)  # Re-display the buttons
        display(classification_selection)

    regression_button.on_click(on_regression_button_clicked)
    classification_button.on_click(on_classification_button_clicked)
    
    regression_target = widgets.Dropdown(
    options=[col for col in dataset.columns if dataset[col].dtype in ['int64', 'float64']],
    description='Actual:',
    )
    regression_prediction = widgets.Dropdown(
        options=[col for col in dataset.columns if dataset[col].dtype in ['int64', 'float64']],
        description='Predicted:',
    )

    def generate_regression_report(b):
        y_true = current_dataset[regression_target.value]
        y_pred = current_dataset[regression_prediction.value]
        results = calculate_regression_metrics(y_true, y_pred)
        display(pd.DataFrame(results))
        save_regression_results_to_excel(results, 'train' if dataset_type.value == 'train' else 'test')

    regression_generate_button = widgets.Button(description='Generate Regression Report', tooltip="Generate the regression report.")
    regression_generate_button.on_click(generate_regression_report)

    regression_selection = widgets.VBox([regression_target, regression_prediction, regression_generate_button])

    classification_actual = widgets.Dropdown(
        options=[col for col in dataset.columns if dataset[col].dtype in ['int64', 'float64']],
        description='Actual:',
    )
    classification_prediction_prob = widgets.Dropdown(
        options=[col for col in dataset.columns if dataset[col].dtype in ['int64', 'float64']],
        description='Predicted Probabilities:',
    )
    classification_amount = widgets.Dropdown(
        options=[col for col in dataset.columns if dataset[col].dtype in ['int64', 'float64']],
        description='Amount:',
    )

    threshold_slider = widgets.FloatSlider(
        value=0.5, min=0.0, max=1.0, step=0.01, description='Threshold:',
    )

    def generate_classification_report(b):
        y_true = current_dataset[classification_actual.value]
        probabilities = current_dataset[classification_prediction_prob.value]
        amount = current_dataset[classification_amount.value]

        threshold = threshold_slider.value
        y_pred = (probabilities >= threshold).astype(int)

        results, confusion = calculate_classification_metrics(y_true, y_pred)

        display(widgets.HTML('<h4>Model Metrics</h4>'))
        display(pd.DataFrame(results))
        display(pd.DataFrame(confusion, columns=['Predicted 0', 'Predicted 1'], index=['Actual 0', 'Actual 1']))

        business_metrics = {'Metric': [], 'Value': []}  # Placeholder for business metrics

        save_classification_results_to_excel(results, confusion, business_metrics, 'train' if dataset_type.value == 'train' else 'test')

    classification_generate_button = widgets.Button(description='Generate Model Metrics', tooltip="Generate the model metrics report.")
    classification_generate_button.on_click(generate_classification_report)

    gdr_slider = widgets.FloatSlider(
        value=0.1, min=0.0, max=1.0, step=0.01, description='GDR:',
    )

    def generate_business_metrics(b):
        y_true = current_dataset[classification_actual.value]
        probabilities = current_dataset[classification_prediction_prob.value]
        amount = current_dataset[classification_amount.value]

        gdr_threshold = gdr_slider.value
        business_metrics = calculate_business_metrics(y_true, probabilities, amount, gdr_threshold)

        display(widgets.HTML('<h4>Business Metrics</h4>'))
        display(pd.DataFrame(business_metrics))

        save_classification_results_to_excel({}, [], business_metrics, 'train' if dataset_type.value == 'train' else 'test')

    business_generate_button = widgets.Button(description='Generate Business Metrics', tooltip="Generate the business metrics report.")
    business_generate_button.on_click(generate_business_metrics)

    classification_selection = widgets.VBox([
        classification_actual,
        classification_prediction_prob,
        classification_amount,
        threshold_slider,
        widgets.Label("If unsure, use the default threshold of 0.5."),
        classification_generate_button,
        gdr_slider,
        business_generate_button
    ])

    train_buttons = widgets.VBox([regression_button, classification_button])
    test_buttons = widgets.VBox([regression_button, classification_button])


    display(dataset_type)


In [2]:
# Example Usage
df = pd.DataFrame({
    'r_actual': [3, 5, 2, 7, 8],
    'r_predicted': [2.5, 5.1, 2.0, 7.2, 7.8],
    'c_actual': [1, 0, 0, 1, 1],
    'c_predicted': [0.78, 0.39, 0.21, 0.63, 0.15],
    'trans_amt': [200, 300, 400, 100, 700]
})

In [3]:
create_widgets_for_metrics(df)

Dropdown(description='Dataset Type:', index=2, options=(('Select One', 'select_one'), ('Train', 'train'), ('Te…

Button(description='Regression Metrics', style=ButtonStyle(), tooltip='Generate metrics for regression tasks.'…

Button(description='Classification Metrics', style=ButtonStyle(button_color='lightblue'), tooltip='Generate me…

HTML(value='<h4>Model Metrics</h4>')

,Metric,Value
0,Precision,1.000000
1,Recall,0.333333
2,F1-Score,0.500000
3,AUC ROC Score,0.666667
4,GINI,0.333333


,Predicted 0,Predicted 1
Actual 0,2,0
Actual 1,2,1


Classification results saved to classification_results.xlsx, sheet: test


HTML(value='<h4>Business Metrics</h4>')

,Metric,Value
0,TDR,0.666667
1,VDR,0.300000


Classification results saved to classification_results.xlsx, sheet: test
